In [5]:
import cv2
import numpy as np

# ----------------------------
# 1. อ่านภาพ
# ----------------------------
img = cv2.imread("rice.jpg")
if img is None:
    print("Image not found")
    exit()

original = img.copy()

# ----------------------------
# 2. White Balance
# ----------------------------
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l,a,b = cv2.split(lab)

l = l.astype(np.float32)
a = a.astype(np.float32)
b = b.astype(np.float32)

avg_a = np.mean(a)
avg_b = np.mean(b)

a -= ((avg_a - 128) * (l / 255.0) * 1.1)
b -= ((avg_b - 128) * (l / 255.0) * 1.1)

lab = cv2.merge([
    np.clip(l,0,255).astype(np.uint8),
    np.clip(a,0,255).astype(np.uint8),
    np.clip(b,0,255).astype(np.uint8)
])

img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# ----------------------------
# 3. Gray + CLAHE + Bilateral
# ----------------------------
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

clahe = cv2.createCLAHE(2.0,(8,8))
gray = clahe.apply(gray)

blur = cv2.bilateralFilter(gray,9,75,75)

# ----------------------------
# 4. Threshold
# ----------------------------
thresh = cv2.adaptiveThreshold(
    blur,255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    51,2
)

# ----------------------------
# 5. Morphology
# ----------------------------
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel,2)
sure_bg = cv2.dilate(opening,kernel,3)

# ----------------------------
# 6. Distance transform
# ----------------------------
dist = cv2.distanceTransform(opening,cv2.DIST_L2,5)
_, sure_fg = cv2.threshold(dist,0.4*dist.max(),255,0)

sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# ----------------------------
# 7. Watershed
# ----------------------------
_, markers = cv2.connectedComponents(sure_fg)
markers += 1
markers[unknown==255] = 0
markers = cv2.watershed(original,markers)

# ----------------------------
# 8. HSV สำหรับตรวจสีเสีย
# ----------------------------
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower_yellow = np.array([18,40,120])
upper_yellow = np.array([40,255,255])
yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

# ----------------------------
# 9. Counters
# ----------------------------
good = 0
broken = 0
spoiled = 0
foreign = 0

# ----------------------------
# 10. วิเคราะห์แต่ละเมล็ด
# ----------------------------
for label in np.unique(markers):

    if label <= 1:
        continue

    mask = np.zeros(gray.shape,dtype="uint8")
    mask[markers==label] = 255

    contours,_ = cv2.findContours(
        mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE
    )

    if not contours:
        continue

    c = max(contours,key=cv2.contourArea)
    area = cv2.contourArea(c)

    if area < 120 or area > 8000:
        continue

    x,y,w,h = cv2.boundingRect(c)

    # ---- Shape features ----
    ratio = w/float(h)
    peri = cv2.arcLength(c,True)
    circularity = 4*np.pi*area/(peri*peri+1e-5)

    # ---- สีผิดปกติ ----
    region = yellow_mask[y:y+h,x:x+w]
    yellow_ratio = cv2.countNonZero(region)/(w*h+1e-5)

    label_text = "Good"
    color = (0,255,0)

    # แตก
    if area < 350:
        label_text="Broken"
        color=(0,0,255)
        broken+=1

    # สีเสีย
    elif yellow_ratio > 0.12:
        label_text="Spoiled"
        color=(0,0,255)
        spoiled+=1

    # กลมเกิน = สิ่งแปลกปลอม
    elif circularity > 0.7:
        label_text="Foreign"
        color=(255,0,0)
        foreign+=1

    else:
        good+=1

    cv2.rectangle(original,(x,y),(x+w,y+h),color,2)
    cv2.putText(original,label_text,(x,y-5),
                cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)

# ----------------------------
# 11. Overlay dashboard
# ----------------------------
total = good+broken+spoiled+foreign
quality = (good/total)*100 if total>0 else 0

cv2.rectangle(original,(0,0),(420,120),(0,0,0),-1)

cv2.putText(original,f"Good: {good}",(10,25),
            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),2)

cv2.putText(original,f"Broken: {broken}",(10,50),
            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)

cv2.putText(original,f"Spoiled: {spoiled}",(10,75),
            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)

cv2.putText(original,f"Foreign: {foreign}",(10,100),
            cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)

cv2.putText(original,f"Quality: {quality:.1f}%",(220,60),
            cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)

# ----------------------------
# 12. Show result
# ----------------------------
cv2.imshow("Rice Quality Inspection",original)
cv2.imshow("Thresh",thresh)
cv2.imshow("Yellow mask",yellow_mask)

cv2.waitKey(0)
cv2.destroyAllWindows()